# Module 2: Train and Evaluate a Keras-Based Classifier
---

In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Define paths and parameters
dataset_path = './images_dataSAT/'
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 2
INPUT_SHAPE = (64, 64, 3)

## Task 1: Recursively walk through the dataset_path using the os.walk function to create a list fnames of all image files.

In [ ]:
# Task 1: Create list of all image file names using os.walk
fnames = []

for dirpath, dirnames, filenames in os.walk(dataset_path):
    for filename in filenames:
        # Filter for common image file extensions
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff', '.gif')):
            full_path = os.path.join(dirpath, filename)
            fnames.append(full_path)

# Sort the list for consistency
fnames = sorted(fnames)

print(f"Total number of image files found: {len(fnames)}")
print(f"\nFirst 5 image paths:")
for f in fnames[:5]:
    print(f"  {f}")
print(f"\nLast 5 image paths:")
for f in fnames[-5:]:
    print(f"  {f}")

In [ ]:
# Create ImageDataGenerator with augmentation for training and rescaling for validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Create training generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

## Task 2: Create the validation_generator from dataset_path.

In [ ]:
# Task 2: Create validation_generator
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\nValidation samples: {validation_generator.samples}")
print(f"Class indices: {validation_generator.class_indices}")
print(f"Number of validation batches: {len(validation_generator)}")

In [ ]:
# Build a basic CNN model first to count layers
model = Sequential([
    # Conv Block 1
    Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
    MaxPooling2D((2, 2)),
    
    # Conv Block 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Conv Block 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flatten
    Flatten(),
    
    # Dense layers
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

## Task 3: Count the total number of layers in this CNN model.

In [ ]:
# Task 3: Count total number of layers
total_layers = len(model.layers)

print(f"Total number of layers in the CNN model: {total_layers}")
print(f"\nLayer details:")
print("=" * 50)
for i, layer in enumerate(model.layers):
    print(f"Layer {i+1}: {layer.name} ({layer.__class__.__name__})")

## Task 4: Create and compile a CNN model test_model with four Conv2D layers and five Dense layers.

In [ ]:
# Task 4: Create test_model with 4 Conv2D layers and 5 Dense layers
test_model = Sequential([
    # Conv2D Layer 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=INPUT_SHAPE),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Conv2D Layer 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Conv2D Layer 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Conv2D Layer 4
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Flatten
    Flatten(),
    
    # Dense Layer 1
    Dense(512, activation='relu'),
    Dropout(0.5),
    
    # Dense Layer 2
    Dense(256, activation='relu'),
    Dropout(0.4),
    
    # Dense Layer 3
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    # Dense Layer 4
    Dense(64, activation='relu'),
    Dropout(0.2),
    
    # Dense Layer 5 (Output)
    Dense(1, activation='sigmoid')
])

# Compile the model
test_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print model summary
test_model.summary()

# Count Conv2D and Dense layers
conv_count = sum(1 for layer in test_model.layers if isinstance(layer, Conv2D))
dense_count = sum(1 for layer in test_model.layers if isinstance(layer, Dense))
print(f"\nNumber of Conv2D layers: {conv_count}")
print(f"Number of Dense layers: {dense_count}")
print(f"Total number of layers: {len(test_model.layers)}")

## Task 5: Create the checkpoint callback for the model with maximum accuracy.

In [ ]:
# Task 5: Create checkpoint callback for maximum accuracy
checkpoint_path = 'best_model.keras'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

print(f"Checkpoint callback created successfully.")
print(f"  Filepath: {checkpoint_path}")
print(f"  Monitor: val_accuracy")
print(f"  Mode: max")
print(f"  Save best only: True")

In [ ]:
# Train the model
history = test_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint_callback],
    verbose=1
)

print("\nTraining completed!")

## Task 6: Plot the graph for training loss and validation loss for the model fit.

In [ ]:
# Task 6: Plot training loss and validation loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Training Loss vs Validation Loss
axes[0].plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
axes[0].set_title('Training Loss vs Validation Loss', fontsize=14)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot 2: Training Accuracy vs Validation Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
axes[1].set_title('Training Accuracy vs Validation Accuracy', fontsize=14)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Model Training History', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print final metrics
print(f"\nFinal Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"\nBest Validation Accuracy: {max(history.history['val_accuracy']):.4f} (Epoch {np.argmax(history.history['val_accuracy'])+1})")
print(f"Lowest Validation Loss: {min(history.history['val_loss']):.4f} (Epoch {np.argmin(history.history['val_loss'])+1})")

---
## All 6 tasks completed successfully.